## Hopenhayn and Rogerson (1993, JPE)


#### Environment

- Discrete time.
- individual productivity $z$ follows a 1st order markov process.
- Entrants draw from some distribution $G(z)$.
- firms are subject to an entry cost $c_e$ and a per period fixed cost $c_f$.
- firms' disctount factor is given by $\beta$.
- firms produce $y = f(z, n)$. Assume for simplicity: $y=z f(n) = z n^\alpha$. 
- Firms are subject to adjustment costs: $g(n_t,n_{t-1}) = \tau \max \{0,n_{t-1}-n_t\}$.
- Households have preferences: $u(c,n) = \ln c + An$. They receive wages (normalize to 1), profits from the firms and tax revenues.
- Timing: 
    * incumbents decide to stay or exit / entrants decide to enter or not 
    * incumbents that stay find new $z$, pay $c_f$, make employment/production decisions, receive profits. Exiters receive $-g(0,n_{t-1})$ this period and 0 in all the others.
    * entrants pay $c_e$, draw a $z$. They make their employment decision next period.

#### Incumbent's problem:

An incumbent firm solves the following problem taking prices $p$ and $w=1$ as given: 

$V(z,n; p) = \max_{n'} \{p z f(n') - n'w - pc_f - g(n',n) + \beta \max[-g(0,n'), \mathbb{E}_{z'|z} V(z',n'; p)]\}.$

Let $n'(z,n; p)$ the employment policy function, and $\chi(z,n; p)=\{0,1\}$ the exit policy (1 is exit). Note that the individual firm has next period adjustment cost as: 

$r(z, n) = [1-\chi(z,n)]\mathbb{E}_{s'|s} g(n'(z',n'(z,n)) ,n'(z,n)) + \chi(z,n) g(0, n').$

#### Entry decision
Potential entrants ex-ante identical, after paying the entry cost $c_e$, they draw their productivity $z$. They start with employment $n=0$. Free entry condition implies:
\begin{align}
V^e(p) = \beta\int V(z, 0; p)dG(z) - c_e \leq 0
\end{align}

with equality whenever the mass of entrants is positive, $M > 0$. Differently than the original paper, I will assume that entrants only start production in the next period.


#### Aggregate variables 
Let the firm distribution (of incumbents) to be: $\mu(z,n)$, which is invariant and given by the operator $T(\mu, M; p) = \mu'$.


$Y = \int zf(n'(z, n)) -c_f d\mu(z, n ) dG(z),$

$C = Y,$

$L^d = \int n'(z, n)d\mu(z, n ),$ 

$N = L^d - Mc_e,$

$T = \int  r(z, n) d\mu(z, n ),$

$\Pi = pY - L^d - T - Mc_e.$


#### Household problem
Household problem is a static problem: 
\begin{align}
&\max_{c, N} \ln c - AN \quad \text{ s.t. } \quad pc = N + \Pi + T
\end{align}
Solution of this problem (demand curve) is: $c = 1/Ap$. Labor supply: $N = 1/A - \Pi - T$.

### Numerical Implementation

Below, I will outline the numerical implementation. I start defining packages.


In [50]:
# P.S currently tested on 3.10.7 python version, but these are standard packages and should work in other versions as well.
import numpy as np
#import matplotlib.pyplot as plt  # if you need to plot 
# if you want to speed up the code, change the code and use NUMBA! 
#from numba import njit, jit, guvectorize #  ====> not used in this code
from scipy.optimize import brentq  # root-finding routine
from scipy.stats import norm


In [51]:

def tauchen(N, rho, sigma, mu=0.0, m=3.0):
    s1 = mu/(1 - rho) - m * np.sqrt(sigma**2 / (1 - rho**2))
    sN = mu/(1 - rho) + m * np.sqrt(sigma**2 / (1 - rho**2))
    s = np.linspace(s1, sN, N) # grid values
    step = (s[N-1] - s[0]) / (N - 1)  # evenly spaced grid
    P = np.zeros((N, N))

    for i in range(np.ceil(N/2).astype(int)):
        P[i, 0] = norm.cdf((s[0] - mu - rho*s[i] + step/2) / sigma)
        P[i, N-1] = 1 - norm.cdf((s[N-1] - mu - rho*s[i] - step/2) / sigma)
        for j in range(1, N-1):
            P[i, j] = norm.cdf((s[j] - mu - rho*s[i] + step/2) / sigma) - \
                      norm.cdf((s[j] - mu - rho*s[i] - step/2) / sigma)
    P[np.floor((N-1)/2+1).astype(int):, :] = P[0:np.ceil((N-1)/2).astype(int), :][::-1, ::-1]

    ps = np.sum(P, axis=1)
    P = P / ps[:, np.newaxis] # transition matrix

    # compute invariant distribution of labor process (by iteration)
    inv = np.ones(N) / N # initial guess
    tol=1E-11; maxit=100^4
    for it in range(maxit):
        inv_new = inv @ P
        if np.max(np.abs(inv_new - inv)) < tol: break  
        inv = inv_new # invariant distribution

    return s, P, inv

### Parameters
I set the parameters below. The shock is discretized using the Tauchen algorithm. To keep it simple, I assume the entrants draw their productivity from the invariant distribution of the discretized productivity shock. The employment grid is discretized "by hand". Using employment integers makes our life easier later when computing the model statistics.

In [52]:
def setPar(
    beta = 0.8,   # discount factor
    rho = 0.9,    # persistence
    sigma = 0.2,  # std. deviation
    mu_z = 1.0,   # mean of stochastic proces.
    alpha = 2/3,  # returns to scale
    c_e = 20.0,   # entry cost
    c_f = 20.0,   # fixed cost
    A = 0.01,      # Related to labor supply. Inverse of Dbar in Hopenhayn.
    tau = 0.1,	   # Adjustment costs
    nZ = 101,    # Grid of Z
    w = 1.0      # wages
	#nN = 500      # Will define the employment grid "by hand".    
    ):

	# === SHOCK DISCRETIZATION
    mu_z = mu_z*(1-rho)  # Force mu of the stochastic process to be always mu_z
    gZ, F_trans, invZ = tauchen(nZ, rho, sigma, mu = mu_z, m = 4.0)	
    gZ = np.exp(gZ)

	# === ENTRANTS DISTRIBUTION: Assume they draw from the invariant distribution.	
    Gprob = invZ
	# I will set the top of the invariant distribution to zero, so entrants have lower productivity on average. The adjfactor controls how many points I set to zero.
	#adjfactor = 1.0
	#index_cutoff = ceil(Int, nZ*adjfactor) # everything above this index will be zero
	#Gprob = invZ
	#Gprob[index_cutoff+1:end] .= 0.0
	#Gprob = Gprob/sum(Gprob) # normalize to one	

	# === EMPLOYMENT GRID
    gN = np.hstack([np.arange(0,21,1), np.arange(22,102,2), np.arange(105,505,5), np.arange(550,1050,50), np.arange(1100,5100,100), np.arange(5500,10500,500) ])
    nN = len(gN)

    # Adjustment Grid
    adjust = np.zeros((nN, nN))  # 1st is n 2nd is nprime
    for i in range(nN):
        for iprime in range(nN):
            adjust[i, iprime] = tau * max(0, gN[i] - gN[iprime])

    # Production given grid nodes
    gY = np.zeros((nN, nZ))
    for jn in range(nN):
        for jz in range(nZ):
            gY[jn, jz] = gZ[jz] * gN[jn]**alpha

	# create dictionary with parameters
    param = {}
    param['alpha'] = alpha; param['beta'] = beta; param['F_trans'] = F_trans; param['gZ'] = gZ; param['nZ'] = nZ; param['tau'] = tau;
    param['c_e'] = c_e; param['c_f'] = c_f; param['A'] = A; param['Gprob'] = Gprob; param['w'] = w; param['nN'] = nN; param['gN'] = gN;
    param['adjust'] = adjust; param['gY'] = gY

    return param

param = setPar()
print(param)


{'alpha': 0.6666666666666666, 'beta': 0.8, 'F_trans': array([[0.20443136, 0.05588705, 0.06285918, ..., 0.        , 0.        ,
        0.        ],
       [0.16082424, 0.0488525 , 0.0566335 , ..., 0.        , 0.        ,
        0.        ],
       [0.12378837, 0.04155719, 0.04965488, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.04965488, 0.04155719,
        0.12378837],
       [0.        , 0.        , 0.        , ..., 0.0566335 , 0.0488525 ,
        0.16082424],
       [0.        , 0.        , 0.        , ..., 0.06285918, 0.05588705,
        0.20443136]]), 'gZ': array([ 0.43373312,  0.44994976,  0.46677271,  0.48422465,  0.5023291 ,
        0.52111044,  0.54059399,  0.560806  ,  0.5817737 ,  0.60352536,
        0.62609028,  0.64949887,  0.67378267,  0.69897441,  0.72510803,
        0.75221875,  0.78034309,  0.80951897,  0.83978568,  0.87118403,
        0.90375631,  0.93754642,  0.9725999 ,  1.00896397,  1.04668764,
  

### Value Function of Incumbents
The code below solves the value function of incumbents. For pedagogical purposes, I will use value function with brute for grid search to solve the maximization problem. This is simple but slow.

One can speed up the code in two ways. The first approach is to exploit the monotonicity of the value function. I had some success using the Divide and Conquer algorithm of Gordon and Qiu (2018, QE). This would require some modification of the code below, but nothing else. The second approach is to use a collocation method (splines or Chebyshev). This can substantially reduce the number of nodes and would allow us to solve the maximization problem using a root-finding routine instead of brute force grid search increasing precision. One problem with collocation is that the value function has a kink in the exit decision (discrete choice). The user would have to approximate and iterate on the conditional expectation instead. Simon Mongey has nice notes on this approach. Alternatively, one could include a preference shock to smooth out the kink. The collocation method requires major modifications of the code, including all the other functions but has a lot of potentials.

After we solve for the value function, we recover the policy functions.

In [53]:
def SolveBellmanIncumbents(p_guess, param, print_it=False):
    # unpacking parameters
    beta = param["beta"]; F_trans = param["F_trans"]; gZ = param["gZ"]; c_f = param["c_f"]; nZ = param["nZ"]; 
    nN = param["nN"]; gN = param["gN"]; adjust = param["adjust"];gY = param["gY"]; 
    
    # housekeeping
    tol = 10**-5
    max_iter = 1000
    iter_count = 10

    # initialize VF
    V = -np.ones((nN, nZ))
    Vnext = -np.ones((nN, nZ))
    
    # auxiliary matrices
    scrap_value = -np.repeat(adjust[:, 0][:, np.newaxis], nZ, axis=1)  # nNprime x nZ matrix
    profit_mat = np.zeros((nN, nZ, nN))  # 1st is n 2nd is nprime
    for iN in range(nN):
        for iNp in range(nN):
            for iZ in range(nZ):
                profit_mat[iN, iZ, iNp] = p_guess * gY[iNp, iZ] - gN[iNp] - p_guess * c_f - adjust[iN, iNp]
    
    # ============== VF ITERATION ====
    for it in range(max_iter):
        ExpV = V @ F_trans.T 
        ConV = beta * np.maximum(ExpV, scrap_value)  # nPrime x Z matrix

        for iZ in range(nZ):
            for iN in range(nN):  # loop over the state
                Vnext[iN, iZ] = np.max(profit_mat[iN, iZ, :] + ConV[:, iZ])

        sup = np.max(np.abs(V - Vnext))  # check tolerance
        V[:] = Vnext[:]
        if sup < tol: 
            if print_it: print(f"Iter: {it}. Tol. achieved: {sup:.2E}")
            break
        if (it == max_iter) and print_it: print(f"Max iterations achieved. VF did not converge: {sup:.2E}")
        if (it % iter_count == 0) and print_it: print(f"Iter: {it}. Tol: {sup:.2E}")

    # ============== RECOVER POLICY FUNCTIONS =======    
    ExpV = V @ F_trans.T 
    ConV = beta * np.maximum(ExpV, scrap_value)  # nPrime x Z matrix
    
    exiter = (scrap_value > ExpV).astype(int)  # exit policy (1 == exit)
    
    npi = np.zeros((nN, nZ), dtype=int)  # recover index
    profit = np.zeros((nN, nZ))  # optimal profits
    tax_r = np.zeros((nN, nZ))  # taxes

    for iZ in range(nZ):
        for iN in range(nN):
            npi[iN, iZ] = np.argmax(profit_mat[iN, iZ, :] + ConV[:, iZ])
            profit[iN, iZ] = profit_mat[iN, iZ, npi[iN, iZ]]
            tax_r[iN, iZ] = adjust[iN, npi[iN, iZ]] * (1 - exiter[iN, iZ]) + exiter[iN, iZ] * adjust[iN, 1]
    
    npol = gN[npi]  # employment values

    return V, npi, exiter, npol, profit, tax_r


testVF = SolveBellmanIncumbents(1.0, param, print_it = True)
#print(testVF[0])
#print(testVF[3])
#print(testVF[5])

Iter: 0. Tol: 1.02E+03
Iter: 10. Tol: 2.47E+00
Iter: 20. Tol: 3.20E-02
Iter: 30. Tol: 5.16E-04
Iter: 40. Tol. achieved: 8.49E-06


### Solving for Prices
To solve for the equilibrium price, we use the free entry condition. We write a function that takes a price and spits out the $V_e(p)$. In equilibrium, we should have $V_e (p)$ = 0.

There is also a function that iterates on the cost of entry (assuming $ p = 1$). This can be useful when the model is calibrated.

In [54]:
def solve_price(param, solvefor_ce=False, print_it=False):
    if not solvefor_ce:  # === SOLVE FOR EQ. USING PRICES
        def entry(p_guess):
            V, _, _, _, _, _ = SolveBellmanIncumbents(p_guess, param)
            excess_entry = sum(V[1,:] * param['Gprob']) - param['c_e']
            if print_it: print("Excess entry: ", excess_entry)
            return excess_entry

        p0 = 0.1; p1 = 4.0 # guess: lower and upper bound. Might have to change for diff. parameters
        p = brentq(entry, p0, p1, xtol = 0.01)
        c_e = param['c_e'] # not important, but must define c_e otherwise it throws an error
        V, npi, exiter, npol, profit, tax_r = SolveBellmanIncumbents(p, param)

    elif solvefor_ce:  # === ASSUME P=1 AND SOLVE FOR COST OF ENTRY -> this is faster and impose entry/exit but ce is chosen endogenously
        p = 1
        V, npi, exiter, np, profit, tax_r = SolveBellmanIncumbents(p, param)
        def entry_ce(c_e):
            excess_entry = sum(V[1,:] * param['Gprob']) - c_e
            if print_it: print("Excess entry: ", excess_entry)
            return excess_entry

        c0 = -120; c1 = 120 # guess: lower and upper bound. Might have to change for diff. parameters
        c_e = brentq(entry_ce, c0, c1, xtol = 0.001)

    return p, V, exiter, npi, npol, tax_r, profit, c_e

sol_ptest =solve_price(param, print_it=True) 
sol_ptest

Excess entry:  -22.09960960540634
Excess entry:  1921.3209164125224
Excess entry:  -22.98233692022623
Excess entry:  139.87583348698516
Excess entry:  -27.931105849052095
Excess entry:  -17.996644107769953
Excess entry:  31.68592590066632
Excess entry:  -5.390532081120778
Excess entry:  0.6173364327614266
Excess entry:  -0.0423051420402345


(1.4433967740614884,
 array([[  -28.86793548,   -28.86793548,   -28.86793548, ...,
          3650.5618276 ,  3981.98698634,  4335.86740381],
        [  -28.96793548,   -28.96793548,   -28.96793548, ...,
          3650.5618276 ,  3981.98698634,  4335.86740381],
        [  -29.06793548,   -29.06793548,   -29.06793548, ...,
          3650.5618276 ,  3981.98698634,  4335.86740381],
        ...,
        [ -928.86793548,  -928.86793548,  -928.86793548, ...,
          3090.21304536,  3458.76489649,  3851.12982952],
        [ -978.86793548,  -978.86793548,  -978.86793548, ...,
          3040.21304536,  3408.76489649,  3801.12982952],
        [-1028.86793548, -1028.86793548, -1028.86793548, ...,
          2990.21304536,  3358.76489649,  3751.12982952]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[  0,   0,   0, ..., 17

### Invariant Distribution
We solve for the invariant distribution using a non-stochastic simulation. We apply the usual trick in this class of model: assume $M=1$ and solve for the distribution. Later we can use the linear properties of the distribution to solve for $M$ using the goods market condition.

In [63]:
def invariant_dist(param, solution, print_it=False):
    Gprob = param['Gprob']; F_trans = param['F_trans']; nN = param['nN']; nZ = param['nZ']
    exiter = solution[2]; npi = solution[3]
    
    # housekeeping
    tol = 10**-6
    max_iter = 1000
    iter_count = 20

    inv_dist = np.zeros((nN, nZ))
    inv_dist[0, :] = Gprob # guess
    dsn_next = np.zeros((nN, nZ))
    

    for iter in range(max_iter): 
        dsn_next[:] = 0.0

        for jn in range(nN):
            for jz in range(nZ):
                dsn_next[npi[jn, jz], :] += inv_dist[jn, jz]*F_trans[jz, :]*(1 -exiter[jn, jz])
        dsn_next[0, :] += Gprob

        sup = np.max(np.abs(dsn_next - inv_dist))  # check tolerance
        inv_dist[:] = dsn_next
        
        if sup<tol:
            if print_it: print(f"Iter: {iter}. Tol. achieved: {sup:.2E}")
            break
        if (iter == max_iter) & print_it: print(f"Max iterations achieved. Inv. dist did not converge: {sup:.2E}")
        if (iter%iter_count==0) & print_it: print(f"Iter: {iter}. Tol: {sup:.2E}")
  
    return inv_dist

mu_test = invariant_dist(param, sol_ptest, print_it = True)
mu_test

Iter: 0. Tol: 2.07E-03
Iter: 20. Tol: 1.00E-04
Iter: 40. Tol: 1.04E-05
Iter: 60. Tol: 1.08E-06
Iter: 61. Tol. achieved: 9.69E-07


array([[3.13026335e-05, 1.42156477e-05, 1.97304606e-05, ...,
        1.97304606e-05, 1.42156477e-05, 3.13026335e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

### Solving For $M$
Finally, we can write a function that solves for the invariant distribution, then uses the goods market clearing to solve for the mass of entrants. Then function also computes the aggregate variables.

In [79]:
def solve_m(param, solution, print_it=False):
    gZ = param['gZ']; nN = param['nN']; 
    p = solution[0]; exiter = solution[2]; npol = solution[4]; tax_r = solution[5]; profit = solution[6]

    # Solve for the Invariant Distribution
    mu = invariant_dist(param, solution, print_it=print_it)

    # Solve for the mass of entrants: M
    Ynz = npol**param['alpha'] * gZ.T
    Yone = np.sum((Ynz - param['c_f']) * mu)  # aggregate suppy (without M)
    demand = 1 / (param['A'] * p)
    M = demand / Yone

    # Recover true mu
    mu = M * mu

    # Aggregate variables
    Y = Yone * M
    N = np.sum(npol * mu) + param['c_e'] * M
    T = np.sum(tax_r * mu)
    Pi = np.sum(profit * mu) - param['c_e'] * M
    Pi2 = p * Y - N - T  # compute profit in a different way to check

    # Exit productivity:
    zcut = np.zeros(nN)
    for i in range(nN):
        cut_index = np.argmax(exiter[i, :] == 0)
        zcut[i] = gZ[cut_index]

    return M, mu, zcut, Y, N, T, Pi, Pi2

dist_test = solve_m(param, sol_ptest, print_it = True)
dist_test

Iter: 0. Tol: 2.07E-03
Iter: 20. Tol: 1.00E-04
Iter: 40. Tol: 1.04E-05
Iter: 60. Tol: 1.08E-06
Iter: 61. Tol. achieved: 9.69E-07


(0.19649789264283343,
 array([[6.15090151e-06, 2.79334482e-06, 3.87699393e-06, ...,
         3.87699393e-06, 2.79334482e-06, 6.15090151e-06],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 array([3.38799772, 3.38799772, 3.38799772, 3.38799772, 3.38799772,
        3.38799772, 3.38799772, 3.38799772, 3.38799772, 3.38799772,
        3.38799772, 3.38799772, 3.38799772, 3.38799772, 3.38799772,
        3.38799772, 3.38799772, 3.38799

### Solving the Model
Finally, we can put everything together. The function first solve for the price and policy functions, then the distribution and the mass of entrants. Finally, it calls a function to compute some useful statistics.

In [149]:
def ModelStats(param, sol_price, sol_dist, Printa = True):
    nN = param['nN']; gZ = param["gZ"]; gN = param["gN"]; alpha = param["alpha"]

    p = sol_price[0]; exiter = sol_price[2]; npol = sol_price[4]
    mu = sol_dist[1]; M = sol_dist[0]

    # productivity distribution
    Mu = np.sum(mu)  # mass of operating firms
    pdf_dist = np.sum(mu, axis=0)/Mu
    cdf_dist = np.cumsum(pdf_dist)
    
    # employment distribution by productivity
    agg_emp = np.sum(npol*mu)
    emp_dist = np.sum(npol*mu, axis = 0)/agg_emp

    # misallocation
    MPNs = alpha*gZ * npol**(alpha-1)
    indx_inf = MPNs != np.inf
    MPNv = MPNs[indx_inf]
    muv = mu[indx_inf]

    avg_misalloc = np.sum(np.abs(MPNv - 1/p)*p*muv)/np.sum(muv) *100 # divide by 1/p so we get %
    #max_misalloc = max(np.abs(MPNv - 1/p)*p)

    # stats
    avg_firm_size = agg_emp/Mu
    exit_rate = M/np.sum(mu)
    avg_prod = np.sum(gZ*np.sum(mu, axis=0))/Mu
    
    # firm and employment share by bins
    size_array = np.array([10, 20, 50, 100, 1000])
    share_array = np.zeros((3, len(size_array)))
    share_array[0, :] = size_array
    

    for i in range(len(size_array)):   
        if i == 0:
            indx =  (0.0 <= npol) & (npol <= size_array[i])
        elif i > 0:
            indx =  (size_array[i-1] < npol) & (npol <= size_array[i])

        share_array[1, i] = np.sum(mu[indx])/Mu # firm share
        share_array[2, i] = np.sum(mu[indx]*npol[indx])/agg_emp # emp share
    share_array[1, :] = np.cumsum(share_array[1, :])
    share_array[2, :] = np.cumsum(share_array[2, :])   

    # print 
    if Printa==True:
        print("Model Stats")
        print("Price: ", p)
        print("Avg. Firm Size: ", avg_firm_size)
        print("Exit/entry Rate: ", exit_rate)
        print("Avg. Productivity: ", avg_prod)
        print("Avg. Misallocation (%): ", avg_misalloc)
        print("Agg. Output: ",  sol_dist[3])
        print("Agg. Labor Supply: ", sol_dist[4])
        print("Agg. Tax Revenue: ", sol_dist[5])	
        print("Agg. Profits: ", sol_dist[6])
        print("Mass of Firms: ", Mu)
        print("Mass of Entrants: ", M)
        print("")
        print("Size           ", share_array[0,0], share_array[0,1],share_array[0,2],share_array[0,3],share_array[0,4])
        print("Firm Share     ", share_array[1,0], share_array[1,1],share_array[1,2],share_array[1,3],share_array[1,4])
        print("Emp. Share     ", share_array[2,0], share_array[2,1],share_array[2,2],share_array[2,3],share_array[2,4])

    return (Mu, pdf_dist, emp_dist, avg_firm_size, exit_rate, avg_prod, avg_misalloc, share_array)


test = ModelStats(param, sol_ptest, dist_test)


Model Stats
Price:  1.4433967740614884
Avg. Firm Size:  110.96197645715192
Exit/entry Rate:  0.273874460086194
Avg. Productivity:  4.323618917309473
Avg. Misallocation (%):  4.71793593044019
Agg. Output:  69.28101946536567
Agg. Labor Supply:  83.54232671139113
Agg. Tax Revenue:  1.7543659866358865
Agg. Profits:  14.842773970578138
Mass of Firms:  0.7174743222898238
Mass of Entrants:  0.19649789264283343

Size            10.0 20.0 50.0 100.0 1000.0
Firm Share      0.11821260652059341 0.1974871813114668 0.4323764130980754 0.6747104571160037 0.994543058193878
Emp. Share      0.005376839642505779 0.016503225684097886 0.09352105536698888 0.25206016499514833 0.9197638844283202


/var/folders/c_/2ncn5k1s2279nm7brg1r7xg40000gn/T/ipykernel_27437/1343010119.py:17: RuntimeWarning: divide by zero encountered in power
  MPNs = alpha*gZ * npol**(alpha-1)


In [151]:
def SolveModel(param):
    # Solve the model
    print("Solving for the price...")
    sol_price = solve_price(param, print_it=True)
    print("Solving for the distribution...")

    sol_dist = solve_m(param, sol_price, print_it=True)

    if sol_dist[0] <= 0: print("Warning: No entry, eq. not found.")

    # Compute Stats
    stats = ModelStats(param, sol_price, sol_dist)

    return (sol_price, sol_dist, stats)

solution = SolveModel(param)

Solving for the price...
Excess entry:  -22.09960960540634
Excess entry:  1921.3209164125224
Excess entry:  -22.98233692022623
Excess entry:  139.87583348698516
Excess entry:  -27.931105849052095
Excess entry:  -17.996644107769953
Excess entry:  31.68592590066632
Excess entry:  -5.390532081120778
Excess entry:  0.6173364327614266
Excess entry:  -0.0423051420402345
Solving for the distribution...
Iter: 0. Tol: 2.07E-03
Iter: 20. Tol: 1.00E-04
Iter: 40. Tol: 1.04E-05
Iter: 60. Tol: 1.08E-06
Iter: 61. Tol. achieved: 9.69E-07
Model Stats
Price:  1.4433967740614884
Avg. Firm Size:  110.96197645715192
Exit/entry Rate:  0.273874460086194
Avg. Productivity:  4.323618917309473
Avg. Misallocation (%):  4.71793593044019
Agg. Output:  69.28101946536567
Agg. Labor Supply:  83.54232671139113
Agg. Tax Revenue:  1.7543659866358865
Agg. Profits:  14.842773970578138
Mass of Firms:  0.7174743222898238
Mass of Entrants:  0.19649789264283343

Size            10.0 20.0 50.0 100.0 1000.0
Firm Share      0.11

/var/folders/c_/2ncn5k1s2279nm7brg1r7xg40000gn/T/ipykernel_27437/1343010119.py:17: RuntimeWarning: divide by zero encountered in power
  MPNs = alpha*gZ * npol**(alpha-1)
